In [3]:
BINGO = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7"""

In [177]:
import dataclasses
from typing import List, Optional
import numpy as np

GRID_SIZE = (5, 5)

@dataclasses.dataclass
class BingoGrid:    
    grid: np.array
    marked: np.array
        
    def mark(self, n):
        self.marked[self.grid == n] = True
    
    def mark_and_check(self, n) -> Optional[int]:
        self.mark(n)
        return self.score(n) if self.wins() else None
    
    def score(self, n) -> int:
        return n * self.grid[~self.marked].sum()
        
    def wins(self) -> bool:
        return self.marked.all(axis=0).any() or self.marked.all(axis=1).any()
        
    @staticmethod
    def from_str(grid_str) -> 'BingoGrid':
        grid = np.array([int(i) for i in  grid_str.replace('\n', ' ').strip().split()], 
                        dtype=int).reshape(GRID_SIZE)
        marked = np.zeros(GRID_SIZE, dtype=bool)
        return BingoGrid(grid, marked)

@dataclasses.dataclass
class Bingo:
    numbers: List[int]
    boards: List[BingoGrid]

In [178]:
def parse_bingo(bingo: str):
    numbers, grids = bingo.split('\n\n', 1)
    numbers = [int(i) for i in numbers.split(',')]
    grids = [BingoGrid.from_str(grid) for grid in grids.split('\n\n')]
    return numbers, grids

In [185]:
def first_winning_score(bingo: str):
    numbers, grids = parse_bingo(bingo)
    for n in numbers:
        for g in grids:
            if score := g.mark_and_check(n):
                return score
            
def last_winning_score(bingo: str):
    numbers, grids = parse_bingo(bingo)
    for n in numbers:
        ws, ls = [], []
        for g in grids:
            (ws if g.mark_and_check(n) else ls).append(g)
        # Return the last winner if there are no losers left.
        if not ls:
            return ws[-1].score(n)
        grids = ls

In [186]:
print(first_winning_score(BINGO))
print(last_winning_score(BINGO))

4512
1924


In [187]:
with open('../data/day04.txt') as infile:
    bingo_str = infile.read()
    print('[p1] first winning score:', first_winning_score(bingo_str))
    print('[p2] last winning score:', last_winning_score(bingo_str))

[p1] first winning score: 49860
[p2] last winning score: 16356
